In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError, MeanAbsolutePercentageError, RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, RMSprop
import pandas as pd
import json
with open("rec_2024-08-23T21_09_27.json", "r") as f:
  cnt = f.read()
  data = json.loads(cnt)
"""with open("rec_2024-08-18T16_53_11.json", "r") as f:
    cnt = f.read()
    val_data = json.loads(cnt)
"""
prec = 3
data.keys()

dict_keys(['checkpoints', 'distances', 'records'])

In [18]:
df = pd.DataFrame(data["records"])
df = df.drop_duplicates().reset_index(drop=True)
df

,InputA,InputS,Sensor1,Sensor10,Sensor11,Sensor12,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Speed
0,0,0,1.0,0.440000,0.440000,1.0,1.0,1.0,0.366667,0.440000,0.440000,0.440000,0.440000,0.440000,0.000000
1,1,0,1.0,0.440087,0.440087,1.0,1.0,1.0,0.366596,0.439913,0.439931,0.439931,0.440000,0.440000,4.166667
2,1,0,1.0,0.440174,0.440174,1.0,1.0,1.0,0.366524,0.439826,0.439861,0.439861,0.440000,0.440000,8.333294
3,1,0,1.0,0.440260,0.440261,1.0,1.0,1.0,0.366453,0.439740,0.439792,0.439792,0.440000,0.440000,12.499823
4,1,0,1.0,0.440347,0.440347,1.0,1.0,1.0,0.366382,0.439653,0.439723,0.439723,0.440001,0.440001,16.666195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220140,1,0,1.0,0.297506,0.651207,1.0,1.0,1.0,0.243808,0.530112,0.290034,0.552808,0.297453,0.589087,1000.000000
220141,1,0,1.0,0.300128,0.648200,1.0,1.0,1.0,0.246277,0.527485,0.292919,0.550099,0.300232,0.586288,1000.000000
220142,1,0,1.0,0.302751,0.645192,1.0,1.0,1.0,0.248746,0.524858,0.295804,0.547389,0.303011,0.583489,1000.000000
220143,1,0,1.0,0.305374,0.642185,1.0,1.0,1.0,0.251215,0.522231,0.298690,0.544680,0.305791,0.580690,1000.000000


In [19]:
df.loc[:, "Speed"] = df["Speed"]/df["Speed"].max()
for c in df.columns:
    df.loc[:, c] = np.round(df[c], prec)
df

,InputA,InputS,Sensor1,Sensor10,Sensor11,Sensor12,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Speed
0,0,0,1.0,0.440,0.440,1.0,1.0,1.0,0.367,0.440,0.440,0.440,0.440,0.440,0.000
1,1,0,1.0,0.440,0.440,1.0,1.0,1.0,0.367,0.440,0.440,0.440,0.440,0.440,0.004
2,1,0,1.0,0.440,0.440,1.0,1.0,1.0,0.367,0.440,0.440,0.440,0.440,0.440,0.008
3,1,0,1.0,0.440,0.440,1.0,1.0,1.0,0.366,0.440,0.440,0.440,0.440,0.440,0.012
4,1,0,1.0,0.440,0.440,1.0,1.0,1.0,0.366,0.440,0.440,0.440,0.440,0.440,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220140,1,0,1.0,0.298,0.651,1.0,1.0,1.0,0.244,0.530,0.290,0.553,0.297,0.589,1.000
220141,1,0,1.0,0.300,0.648,1.0,1.0,1.0,0.246,0.527,0.293,0.550,0.300,0.586,1.000
220142,1,0,1.0,0.303,0.645,1.0,1.0,1.0,0.249,0.525,0.296,0.547,0.303,0.583,1.000
220143,1,0,1.0,0.305,0.642,1.0,1.0,1.0,0.251,0.522,0.299,0.545,0.306,0.581,1.000


In [20]:
X_train = df.drop(["InputA", "InputS"], axis=1).values
y_train = df[["InputA", "InputS"]].values

In [6]:
nn = Sequential()
nn.add(Dense(units = X_train.shape[1], activation="relu"))
nn.add(Dense(units=128, activation="relu"))
nn.add(Dense(units=64, activation="relu"))
nn.add(Dense(units=32, activation="relu"))
nn.add(Dense(units=16, activation="relu"))
nn.add(Dense(units=8, activation="relu"))
#nn.add(Dense(units=4, activation="relu"))
nn.add(Dense(units=2))

nn.compile(optimizer=Adam(learning_rate=0.00001)
, loss='mean_squared_error')

In [32]:
nn.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 31s 4ms/step - loss: 0.4933
Epoch 2/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 0.3402
Epoch 3/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3119
Epoch 4/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.2914
Epoch 5/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.2753
Epoch 6/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.2662
Epoch 7/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.2579
Epoch 8/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.2532
Epoch 9/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.2469
Epoch 10/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.2434
Epoch 11/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.2388
Epoch 12/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.2363
Epoch 13/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.2319
Epoch 14/100
6880/6880 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 0.2303
E

In [7]:
#Testing function
ath = 0
sth = 0
def cmp(y1, y2):
    global ath, sth
    a1 = y1[0]
    a2 = y2[0]
    s1 = y1[1]
    s2 = y2[1]
    return [(a1>0 and a2>ath) or (a1<0 and a2<=-ath) or (a1==0 and -ath<a2<ath),
           (s1>0 and s2>sth) or (s1<0 and s2<=-sth) or (s1==0 and -sth<s2<sth)]


In [8]:
# testing the model on training data
y_pred_train = nn.predict(X_train)
for i in range(11):
        ath = i/10
        sth = i/10
        results = np.array(list(map(cmp, y_train, y_pred_train)))
        print(ath," -> ", 100 * results[:, 0].sum()/len(results), 100 * results[:, 1].sum()/len(results))

6880/6880 ━━━━━━━━━━━━━━━━━━━━ 32s 4ms/step
0.0  ->  15.810488541643007 30.66206363987372



KeyboardInterrupt



In [ ]:
nn.predict(X_train[0].reshape(1, -1))

In [36]:
nn.save("nn.keras")

In [30]:
inp = Input((X_train.shape[1], ))
br1 = Dense(units=128, activation="relu")(inp)
br1 = Dense(units=64, activation="relu")(br1)
br1 = Dense(units=16, activation="relu")(br1)
br1 = Dense(units=4, activation="relu")(br1)
br1 = Dense(units=1, activation="relu")(br1)

br2 = Dense(units=128, activation="relu")(inp)
br2 = Dense(units=64, activation="relu")(br2)
br2 = Dense(units=16, activation="relu")(br2)
br2 = Dense(units=4, activation="relu")(br2)
br2 = Dense(units=1, activation="relu")(br2)

nn2 = Model(inp, [br1, br2])
nn2.compile(optimizer=Adam(learning_rate=0.00001)
, loss='mean_squared_error')

In [ ]:
nn2.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
